In [ ]:
!pip install -qqq -U wandb --progress-bar off
import wandb
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

wb_token = userdata.get('wandb')
wandb.login(key=wb_token)

In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git --progress-bar off
!pip install -q -U git+https://github.com/huggingface/accelerate.git --progress-bar off
!pip install datasets evaluate --progress-bar off
!pip install -q -U bitsandbytes --progress-bar off
!pip install -q -U git+https://github.com/huggingface/peft.git --progress-bar off

In [ ]:
from transformers import AutoTokenizer

base_model_id = "microsoft/Phi-3-mini-4k-instruct"
# eval tokenizer does not have eos token and padding
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
    use_fast=False,
    #truncate=True,
    #padding_side="left", # https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa
)
#tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from datasets import load_dataset
import evaluate

split = "validation"#"test"#
mrqa_eval = load_dataset("enriquesaou/mrqa-squadded-sample", split=split)

In [ ]:
# source: https://github.com/mrqa/MRQA-Shared-Task-2019/blob/master/mrqa_official_eval.py

import string
import re
import json
import gzip
from collections import Counter

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def read_predictions(prediction_file):
    with open(prediction_file) as f:
        predictions = json.load(f)
    return predictions


def read_answers(gold_file):
    answers = {}
    with gzip.open(gold_file, 'rb') as f:
        for i, line in enumerate(f):
            example = json.loads(line)
            if i == 0 and 'header' in example:
                continue
            for qa in example['qas']:
                answers[qa['qid']] = qa['answers']
    return answers


def evaluate_predictions(answers, predictions, skip_no_answer=False):
    f1 = exact_match = total = 0

    for qid, ground_truths in answers.items():
        if qid not in predictions:
            if not skip_no_answer:
                message = 'Unanswered question %s will receive score 0.' % qid
                print(message)
                total += 1
            continue
        total += 1
        prediction = predictions[qid]
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

In [ ]:
def format_cqa(context, question):
    return "Answer the question extracting from the context below.\nContext: " + context + "\nQuestion: " + question + "\nAnswer: "

In [ ]:
def format_phi3(context, question): # not good enough
    cq = "Answer the question extracting from the context below.\nContext: " + context + "\nQuestion: " + question
    return "<|user|>\n" + cq + "<|end|>\n<|assistant|>"

In [ ]:
def tokenize_and_generate(test_model, prompt, new_tokens=16):
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to('cuda')
    with torch.no_grad():
        outputs = test_model.generate(**inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=new_tokens)
        answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    return answer

In [ ]:
models_to_evaluate = ["enriquesaou/phi-3-mrqa"]#[base_model_id]

In [ ]:
import torch
from tqdm.auto import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm


new_tok = 5

for model_id in models_to_evaluate:

    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        trust_remote_code=True,
        device_map="auto",
        torch_dtype=torch.float16, # to fit into T4
    )

    # load peft if not base
    if model_id != base_model_id:
      # note that base_model may be modified in place
      model_for_eval = PeftModel.from_pretrained(base_model, model_id)
    else: model_for_eval = base_model

    model_for_eval.eval()

    all_predictions = {}
    for example in tqdm(mrqa_eval):
      prompt = format_cqa(example['context'], example['question'])
      if len(prompt) > 5000: continue # OOM fix
      outs = tokenize_and_generate(model_for_eval, prompt, new_tokens=new_tok)
      outs = outs.replace(prompt, '')
      outs = outs.split('Answer:')[1] if 'Answer:' in outs else outs
      all_predictions[example['id']] = outs.strip()


    # compute metrics
    answers = mrqa_eval.to_dict()
    answers = {id: aws['text'] for id, aws in zip(answers['id'], answers['answers'])}
    metrics = evaluate_predictions(answers, predictions=all_predictions)

    print(model_id, split, json.dumps(metrics))

    """
    for k in answers.keys():
        print(all_predictions[k], answers[k])
        metrics = evaluate_predictions({k: answers[k]}, {k: all_predictions[k]})
        print(json.dumps(metrics))
    """


In [ ]:
# with phi3 format
predis = {}
for k in answers.keys():
        pred = all_predictions[k].split('<|assistant|>')[1]
        print(pred, answers[k])
        #metrics = evaluate_predictions({k: answers[k]}, {k: pred})
        #print(json.dumps(metrics))
        predis[k] = pred

metrics = evaluate_predictions(answers, predictions=predis)
print(model_id, split, json.dumps(metrics))